# Step-by-step guide to build MuSIC v1

Before starting, please install relevant packages following guidelines in the **Dependency** section of GitHub README.

## Calibrate protein-protein distance and proximity from Gene Ontology


In [1]:
%%bash
python /cellar/users/y8qin/Data2/deepLoc/MuSIC/calibrate_pairwise_distance.py \
--protein_file /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/MuSIC_proteins.txt \
--outprefix /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/output/test

Calibration Function:
log10(D) = 1.05 * log10(C) + -0.14
R2 = 0.89

Analyzing 661 proteins:
DMAP1, COLEC12, FBXL14, ASB13, CD72, EYA3, OAZ2, CCNDBP1, CDKN2B, PICK1, ZNF232, ZNF212, FKBPL, TRAPPC5, LMX1B, SUCLA2, DPYSL5, ORC2, TBKBP1, HSD17B6, RRN3, AGK, CUEDC2, KLHL15, UTP18, SEPT1, NAT14, PRDM16, KIDINS220, SCML1, DACH1, MCOLN2, PATZ1, TP73, ZNF224, EEF1D, WASF2, CREB5, WDR90, PRPSAP1, CASC3, ZDBF2, PHLPP2, MRPL20, SALL2, SMARCD2, CTU2, SFPQ, HUS1B, MTOR, RPL7L1, MAD2L2, SMAD9, GPR50, FHIT, RNF220, FAM160A2, ZNF416, SLC35F2, COL2A1, SDC2, GTF3C6, XKR8, RPL28, CASP8AP2, QPCT, FUZ, ZNF280B, NOS1AP, MCAT, SLC35F1, CTR9, LIN28B, SEPHS2, DNAJA3, CDO1, NASP, TAF11, ANKRD54, NSMCE4A, CLASRP, SYN2, WBP11, CAMK1, MORF4L1, TNNI3, CCDC130, TWIST2, ZNF580, MAP1S, CHEK1, ZNF32, YARS2, CDKAL1, LBR, MFF, UBE2M, GPS2, INO80C, SFMBT2, XPO1, HSF2, KRBA1, FAM184A, CIR1, POMT1, PCDH19, ZNF266, LRRC34, TOR1A, SGSM3, BAP1, C8orf34, DDB1, FYN, PSMG4, MED25, KIAA2013, SKI, AKAP17A, ARVCF, IRF2BP1, CDC40, THA

100%|█████████▉| 1601/1602 [00:24<00:00, 65.20it/s]


## Random forest prediction of protein distances

Each random forest regressor in the original MuSIC study was trained with ~1M samples consisted of 2060 input features, requiring ~1 day and >100 Gb memory with 24 threads. As a demo, code below will generate two set of embeddings for image embeddings (MuSIC used six) and create 1000 samples for training.

In [2]:
%%bash
python random_forest_samples.py \
--outprefix /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/output/test \
--protein_file /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/MuSIC_proteins.txt \
--emd_files /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/IF_image_embedding.csv \
/cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/APMS_embedding.MuSIC.csv \
--emd_label IF_emd APMS_emd \
--num_set 2 auto \
--n_samples 1000 

Formatting IF_emd into 2 training sets using file /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/IF_image_embedding.csv ...
Number of embeddings per protein ranges from 2 to 6.
Format IF_emd into 2 training sets...
Start calculating pairwise protein similarity for IF_emd with training set 1 ...
Start calculating pairwise protein similarity for IF_emd with training set 2 ...
Assess distinctness among different training sets generated (Jaccard)...


Formatting APMS_emd into auto training sets using file /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/APMS_embedding.MuSIC.csv ...
Number of embeddings per protein ranges from 1 to 1.
Format APMS_emd into 1 training sets...
Start calculating pairwise protein similarity for APMS_emd with training set 1 ...


Split protein pairs into training and testing sets for 5-fold cross validation...
Sampled 880 training samples for fold 1
Sampled 880 training samples for fold 2
Sampled 900 training samples for fold 3
Sampled 890 training samples for 

In [5]:
%%bash
for ((fold = 1; fold <= 5; fold++))
do
    for ((IF_set = 1; IF_set <= 2; IF_set++))
    do
        python run_random_forest.py \
        --outprefix /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/output/test \
        --fold $fold \
        --emd_label IF_emd APMS_emd \
        --train_set $IF_set 1 \
        --n_jobs 60;
    done
done

Trained model will be saved in /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/output/test.IF_emd_1_APMS_emd_1.RF_maxDep_30_nEst_1000.fold_1.pkl
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=60,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
... loaded training data
Start training random forest regressor on 880 samples with 2060 total features...
... finished training
... saved trained random forest regressor

Start predicting test data...
... loaded 36181 test samples
... finished predicting test samples
Pearson r: 0.21793454183019872

Start predicting gene pairs without specific GO annotations...

=== finished! ===
Trained model will be saved in /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/output/

/cellar/users/y8qin/anaconda3/envs/music/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/cellar/users/y8qin/anaconda3/envs/music/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/cellar/users/y8qin/anaconda3/envs/music/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/cellar/users/y8qin/anaconda3/envs/music/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.p

In [6]:
%%bash
python random_forest_output.py --outprefix /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/output/test

=== finished! ===


The predicted protein-protein proximity for all pairs of the given protein is saved in **$outprefix_predicted_proximity.txt** file. Because the proximity predicted from running above code is only a demo with 1000 training samples, we will run the following sections using **Examples/MuSIC_predicted_proximity.txt** file instead. Since CliXO performs community detection by thresholding the input network, one can boost performance and memory required by rounding the number of decimals of the predicted proximity.

## Pan-resolution community detection

To perform pan-resolution community detection as in MuSIC, please install [CliXO v1.0](https://github.com/fanzheng10/CliXO-1.0), [alignOntology](https://github.com/mhk7/alignOntology), and [DDOT](https://github.com/michaelkyu/ddot). Note for DDOT, the requirements are satisfied, but users need to follow instructions in section *Install the ddot Python package*. Also note that DDOT has alignOntology in `/ddot/alignOntology` folder. If user has trouble installing  alignOntology from GitHub, user can use path to alignOntology in DDOT for the `--path_to_alignOntology` parameter in the community detection section.

In [7]:
%%bash
python community_detection.py \
--outprefix /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/output/test \
--path_to_clixo /cellar/users/y8qin/Modules/CliXO \
--clixo_i /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/MuSIC_predicted_proximity.txt \
--clixo_a 0.01 --clixo_b 0.5 --clixo_m 0.008 --clixo_z 0.05 --min_diff 2 \
--path_to_alignOntology /cellar/users/y8qin/Modules/alignOntology-master \
--predict_nm_size --keep_all_files

# Note that CliXO can take ~7 hours for MuSIC. It's recommended to run the below command line in background
bash /cellar/users/y8qin/Data2/deepLoc/MuSIC/Examples/output/test.sh

Process is interrupted.
